In [ ]:
import os
import numpy as np
import pandas as pd
import librosa
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import random

In [ ]:
real_audio_path = r"/content/drive/MyDrive/MIS 545/REAL"
fake_audio_path = r"/content/drive/MyDrive/MIS 545/FAKE"
sample_rate = 22050
segment_duration = 3
overlap = 1

def augment_audio_add_noise(audio, noise_factor=0.005):
    noise = np.random.randn(len(audio))
    augmented_audio = audio + noise_factor * noise
    return augmented_audio

def load_audio_files(folder_path, label, sr=sample_rate, augment=False):
    data = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if filename.endswith('.wav'):
            audio, _ = librosa.load(file_path, sr=sr)

            if augment and random.random() < 0.5:
                audio = augment_audio_add_noise(audio, noise_factor=0.005)

            data.extend(segment_and_extract_features(audio, label, sr))
    return data

def segment_and_extract_features(audio, label, sr):

    features = []
    segment_samples = int(segment_duration * sr)
    overlap_samples = int(overlap * sr)
    num_segments = (len(audio) - segment_samples) // (segment_samples - overlap_samples) + 1

    for i in range(num_segments):
        start = i * (segment_samples - overlap_samples)
        end = start + segment_samples
        segment = audio[start:end]

        mfccs = librosa.feature.mfcc(y=segment, sr=sr, n_mfcc=13)
        chroma = librosa.feature.chroma_stft(y=segment, sr=sr)
        spectral_centroid = librosa.feature.spectral_centroid(y=segment, sr=sr)
        spectral_bandwidth = librosa.feature.spectral_bandwidth(y=segment, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=segment, sr=sr)
        zero_crossing_rate = librosa.feature.zero_crossing_rate(y=segment)

        feature_dict = {
            'mfcc_mean': np.mean(mfccs, axis=1),
            'mfcc_std': np.std(mfccs, axis=1),
            'chroma': np.mean(chroma, axis=1),
            'spectral_centroid': np.mean(spectral_centroid),
            'spectral_bandwidth': np.mean(spectral_bandwidth),
            'rolloff': np.mean(rolloff),
            'zero_crossing_rate': np.mean(zero_crossing_rate),
            'label': label
        }
        features.append(feature_dict)
    return features

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
real_data = load_audio_files(real_audio_path, label='Real', augment=False)
fake_data = load_audio_files(fake_audio_path, label='Fake', augment=True)
all_data = real_data + fake_data

df = pd.DataFrame(all_data)

X = pd.concat([
    pd.DataFrame(df['mfcc_mean'].tolist()),
    pd.DataFrame(df['mfcc_std'].tolist()),
    pd.DataFrame(df['chroma'].tolist()),
    df[['spectral_centroid', 'spectral_bandwidth', 'rolloff', 'zero_crossing_rate']]
], axis=1)
y = df['label']

# Save the DataFrame as a CSV file
df.to_csv('audio_features_data.csv', index=False)

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y = to_categorical(y)

# Ensure all column names are strings
X.columns = X.columns.astype(str)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)